In [1]:
 %%bash
 rm -rf input-marginalization
 git clone https://github.com/ronakdm/input-marginalization.git
 cd input-marginalization
 git pull
 cd ..

Already up to date.


Cloning into 'input-marginalization'...


In [ ]:
import random
import pickle
import numpy as np
import time
import datetime
import random
import torch

In [2]:
try:
    import transformers
except ModuleNotFoundError:
    !pip install transformers
    import transformers

     |████████████████████████████████| 1.9MB 14.0MB/s 
     |████████████████████████████████| 3.2MB 41.3MB/s 
     |████████████████████████████████| 890kB 24.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=b86607edd2c5c4c1d9c578528be32e254a478b7c22b12a1079a152fd9415d22c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import sys
path = ("input-marginalization")
if path not in sys.path:
    sys.path.insert(0, path)

In [48]:
from lm_utils import make_dataloaders
from utils import format_time

In [34]:
LEARNING_RATE = 2e-5
ADAMW_TOLERANCE = 1e-8
BATCH_SIZE = 2
EPOCHS = 2

In [36]:
train_dataloader, valid_dataloader, test_dataloader = make_dataloaders(BATCH_SIZE)

Token indices sequence length is longer than the specified maximum sequence length for this model (2417786 > 1024). Running this sequence through the model will result in indexing errors


4,722 train samples.


Token indices sequence length is longer than the specified maximum sequence length for this model (249887 > 1024). Running this sequence through the model will result in indexing errors


  488 valid samples.


Token indices sequence length is longer than the specified maximum sequence length for this model (286221 > 1024). Running this sequence through the model will result in indexing errors


  559 test samples.


In [42]:
import torch
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print("Running on '%s'." % device)

Running on 'cuda'.


In [43]:
from google.colab import drive
drive.mount('/content/gdrive')
save_dir = "/content/gdrive/My Drive/input-marginalization"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [40]:
def train(
    model,
    epochs,
    train_dataloader,
    validation_dataloader,
    optimizer,
    scheduler,
    save_dir,
    save_filename,
    device,
    seed_val=42,
):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    training_stats = []
    total_t0 = time.time()
    for epoch_i in range(epochs):

        # ========================================
        #               Training
        # ========================================

        print("")
        print("======== Epoch {:} / {:} ========".format(epoch_i + 1, epochs))
        print("Training...")

        t0 = time.time()
        total_train_loss = 0

        model.train()

        for step, batch in enumerate(train_dataloader):

            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print(
                    "  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.".format(
                        step, len(train_dataloader), elapsed
                    )
                )

            inputs,labels=((batch,batch))
            inputs = inputs.to(device)
            labels = labels.to(device)

            model.train()

            outputs = model(
                inputs,labels = labels
            )

            loss = outputs[0]
            #logits = output.logits
            loss.backward()
            total_train_loss += loss.item()

            

            # TODO: See if this is needed.
            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()
            model.zero_grad()

        avg_train_loss = total_train_loss / len(train_dataloader)
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================

        print("")
        print("Running Validation...")

        t0 = time.time()

        

        #total_eval_accuracy = 0
        total_eval_loss = 0.0
        eval_loss = 0.0
        eval_step = 0
        model.eval()

        for batch in validation_dataloader:

            inputs,labels=((batch,batch))
            inputs = inputs.to(device)
            labels = labels.to(device)

            with torch.no_grad():
                outputs = model(
                inputs,labels = labels
                )

                loss = outputs[0]
                #logits = output.logits
                eval_loss += loss.mean().item()
            eval_step += 1
            #total_eval_loss += loss.item()
        eval_loss = eval_loss/eval_step
            #logits = logits.detach().cpu().numpy()
            #label_ids = b_labels.to("cpu").numpy()

            #total_eval_accuracy += flat_accuracy(logits, label_ids)
        perplexity = torch.exp(torch.tensor(eval_loss))
        #avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Perplexity: {0:.2f}".format(perplexity))
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        validation_time = format_time(time.time() - t0)
        print("  Validation Loss: {0:.2f}".format(eval_loss))
        print("  Validation took: {:}".format(validation_time))

        training_stats.append(
            {
                "epoch": epoch_i + 1,
                "Training Loss": avg_train_loss,
                "Valid. Loss": eval_loss,
                "Perplexity":perplexity,
                #"Valid. Accur.": eval_accuracy,
                "Training Time": training_time,
                "Validation Time": validation_time,
            }
        )

    print("")
    print("Training complete!")

    print(
        "Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0))
    )

    # Save the model.
    torch.save(model, f"{save_dir}/{save_filename}.pt")
    pickle.dump(
        training_stats, open(f"{save_dir}/training_stats_{save_filename}.p", "wb")
    )


def test(model, test_dataloader, device, save_dir, save_filename):
    # ========================================
    #               Testing
    # ========================================

    print("")
    print("Testing...")

    t0 = time.time()

    model.eval()

    total_test_accuracy = 0
    total_test_loss = 0
    test_step = 0
    for batch in test_dataloader:

        inputs,labels=((batch,batch))
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(
            inputs,labels = labels
            )

            loss = outputs[0]
            #logits = output.logits
            total_test_loss += loss.mean().item()
        test_step += 1
            #total_eval_loss += loss.item()
    test_loss = total_test_loss/test_step
    perplexity = torch.exp(torch.tensor(test_loss))    
    
    print("  perplexity: {0:.2f}".format(perplexity))
    #avg_test_loss = total_test_loss / len(test_dataloader)
    test_time = format_time(time.time() - t0)
    print("  Test Loss: {0:.2f}".format(test_loss))
    print("  Test took: {:}".format(test_time))

    test_stats = {
        "Test Loss": test_loss,
        "Perplexity":perplexity,
        #"Test Accur.": avg_test_accuracy,
        "Test Time": test_time,
    }
    pickle.dump(test_stats, open(f"{save_dir}/test_stats_{save_filename}.p", "wb"))


In [38]:
from transformers import GPT2LMHeadModel
from transformers import AdamW,get_linear_schedule_with_warmup
model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)

save_filename = "gpt2_wikitext2"

optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps = ADAMW_TOLERANCE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = EPOCHS * BATCH_SIZE * len(train_dataloader))

In [50]:
try:
    train(model, EPOCHS, train_dataloader, valid_dataloader, optimizer, scheduler, save_dir, save_filename, device)
except KeyboardInterrupt:
    print("Graceful Exit")


======== Epoch 1 / 2 ========
Training...
  Batch    40  of  2,361.    Elapsed: 0:00:05.
  Batch    80  of  2,361.    Elapsed: 0:00:09.
  Batch   120  of  2,361.    Elapsed: 0:00:14.
  Batch   160  of  2,361.    Elapsed: 0:00:18.
  Batch   200  of  2,361.    Elapsed: 0:00:23.
  Batch   240  of  2,361.    Elapsed: 0:00:27.
  Batch   280  of  2,361.    Elapsed: 0:00:32.
  Batch   320  of  2,361.    Elapsed: 0:00:37.
  Batch   360  of  2,361.    Elapsed: 0:00:41.
  Batch   400  of  2,361.    Elapsed: 0:00:46.
  Batch   440  of  2,361.    Elapsed: 0:00:50.
  Batch   480  of  2,361.    Elapsed: 0:00:55.
  Batch   520  of  2,361.    Elapsed: 0:01:00.
  Batch   560  of  2,361.    Elapsed: 0:01:04.
  Batch   600  of  2,361.    Elapsed: 0:01:09.
  Batch   640  of  2,361.    Elapsed: 0:01:13.
  Batch   680  of  2,361.    Elapsed: 0:01:18.
  Batch   720  of  2,361.    Elapsed: 0:01:23.
  Batch   760  of  2,361.    Elapsed: 0:01:27.
  Batch   800  of  2,361.    Elapsed: 0:01:32.
  Batch   840  of

In [51]:

test(model, test_dataloader, device, save_dir, save_filename)


Testing...
  perplexity: 21.62
  Test Loss: 3.07
  Test took: 0:00:09
